In [106]:
import torch as th 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import pytorch_lightning as pl 
from pytorch_lightning import Trainer, LightningModule, LightningDataModule

import torchdyn 
from torchdyn.core import NeuralODE

import os
import cv2 
import numpy as np 
import matplotlib.pyplot as plt 

import warnings 
warnings.filterwarnings("ignore")

In [107]:
PATH = r'/Users/suyashsachdeva/Desktop/GyanBhandar/CircuitSolver/'

In [108]:
class CircuitFused(Dataset):
    def __init__(self, datadir, image_size):
        super().__init__()
        self.datadir = datadir
        self.image_size = image_size
        self.images, self.labels = self.datareader()

    def datareader(self):
        X = []
        Y = []
        folders = os.listdir(self.datadir)
        try:
            for folder in folders:
                files = os.listdir(self.datadir+folder)
                for file in files:
                    image = cv2.imread(self.datadir+folder+"/"+file)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    image = cv2.resize(image, (self.image_size[0], self.image_size[1]), cv2.INTER_AREA)/255.0
                    X.append(image.reshape(1, self.image_size[0], self.image_size[1]))
                    Y.append(folder)
        except:
            pass
        return X, Y

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        image = self.images[idx]
        sample = {"Image": image.reshape(1, 64, 64), "Label": label}
        return sample

In [109]:
CircuitDataset = DataLoader(CircuitFused(PATH, (64, 64)), batch_size=100, shuffle=True)

In [110]:
class CNNODEBlock(nn.Module):
    def __init__(self, infilter, outfilter, kernel, moment):
        super(CNNODEBlock, self).__init__()
        self.conv1 = nn.Conv2d(infilter, outfilter, kernel)
        self.act = nn.ReLU()
        self.conv2 = nn.Conv2d(outfilter, infilter, kernel)
        pad = int((kernel-1)/2)
        self.pad = nn.ZeroPad2d(pad)
        self.norm1 = nn.BatchNorm2d(outfilter, momentum=moment)
        self.norm2 = nn.BatchNorm2d(infilter, momentum=moment)

    def forward(self, x):
        x = self.act(self.norm1(self.pad(self.conv1(x))))
        return self.act(self.norm2(self.pad(self.conv2(x))))

In [111]:
class UPBlock(nn.Module):
    def __init__(self, infilter, outfilter, kernel, moment):
        super(UPBlock, self).__init__()
        self.conv1 = nn.Conv2d(infilter, outfilter, kernel)
        self.norm1 = nn.BatchNorm2d(outfilter, momentum=moment)

        self.conv2 = nn.Conv2d(outfilter, outfilter, kernel)
        self.norm2 = nn.BatchNorm2d(outfilter, momentum=moment)

        self.act = nn.ReLU()
        self.pad = nn.ZeroPad2d(int((kernel-1)/2.0))
        self.pool = nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        x = self.act(self.norm1(self.pad(self.conv1(x))))
        x = self.act(self.norm2(self.pad(self.conv2(x))))
        return self.pool(x)

In [112]:
class DenseBlock(nn.Module):
    def __init__(self, filter, dense, drop, classes):
        super(DenseBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dense = nn.Linear(filter, dense)
        self.drop = nn.Dropout(drop)
        self.final = nn.Linear(dense, classes)
    
    def forward(self,x):
        x = self.dense(self.drop(self.pool(x)))
        return F.softmax(self.final(self.drop(F.relu(x))))

In [113]:
class CNNODE(nn.Module):
    def __init__(self, num=3, filter=64, dense=256, classes=15, gf=2, kernel=3, moment=0.9, drop=0.2):
        super(CNNODE, self).__init__()
        self.cnnode = nn.ModuleList([])
        self.upsamp = nn.ModuleList([])
        self.final = DenseBlock(filter*pow(gf, num-1), dense, drop, classes)
        self.conv = UPBlock(1, filter, kernel, moment)
        
        for _ in range(num):
            model = NeuralODE(CNNODEBlock(filter, filter*2, kernel, moment), sensitivity='adjoint', solver='rk4', solver_adjoint='dopri5', atol_adjoint=1e-4, rtol_adjoint=1e-4)
            self.cnnode.append(model)
            self.upsamp.append(UPBlock(filter, filter*2, kernel, moment))
            filter = filter*gf

    def forward(self, x):
        x = self.conv(x)
        for neuralode, neuralnetwork in zip(self.cnnode, self.upsamp):
            x = neuralnetwork(neuralode(x))
        return self.final(x)      

In [114]:
class Learner(LightningModule):
    def __init__(self, t_span:th.tensor, model:nn.Module):
        super(Learner, self).__init__()
        self.model, self.t_span = model, t_span

    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        t_eval, y_hat = self.model(x, self.t_span)
        y_hat = y_hat[-1] # select last point of solution trajectory
        loss = nn.CrossEntropyLoss()(y_hat, y)
        return {'loss': loss}
    
    def configure_optimizers(self):
        return th.optim.Adam(self.model.parameters(), lr=0.0001)
    
    def train_dataloader(self):
        return trainloader

In [115]:
model = CNNODE()
t_span = th.linspace(0, 1, 100)
learn = Learner(t_span, model)

Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.
Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.
Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.


In [116]:
trainer = pl.Trainer(min_epochs=200, max_epochs=300)
trainer.fit(learn, train_dataloaders=CircuitDataset)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type   | Params
---------------------------------
0 | model | CNNODE | 7.9 M 
---------------------------------
7.9 M     Trainable params
0         Non-trainable params
7.9 M     Total params
31.433    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

TypeError: CNNODE.forward() takes 2 positional arguments but 3 were given